## 

In [43]:
!docker-compose up -d db

!docker-compose ps

74621.78s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


[+] Running 2/3
 ✔ Network imdb-project_default       Cr...                                0.0s 
 ✔ Volume imdb-project_postgres_data  Created                              0.0s 
 ⠋ Container imdb-db                  Starting                             0.1s 
[+] Running 3/3
 ✔ Network imdb-project_default       Cr...                                0.0s 
 ✔ Volume imdb-project_postgres_data  Created                              0.0s 
 ✔ Container imdb-db                  Started                              0.1s 


74627.31s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


NAME      IMAGE         COMMAND                  SERVICE   CREATED         STATUS         PORTS
imdb-db   postgres:15   "docker-entrypoint.s…"   db        6 seconds ago   Up 5 seconds   0.0.0.0:5432->5432/tcp, [::]:5432->5432/tcp


In [44]:
import pandas as pd
import gzip
import requests
import shutil
import os
from sys import path
from sqlalchemy import create_engine
from dotenv import load_dotenv
import logging

logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s - %(message)s')

path.append(os.path.realpath("../"))
load_dotenv()

NAME_BASIC_URL = "https://datasets.imdbws.com/name.basics.tsv.gz"

NAMES_ZIP_FILE = "name.basics.tsv.gz"
NAMES_FILE = "name.basics.tsv"

if not os.path.isfile(NAMES_FILE):
    response = requests.get(NAME_BASIC_URL, stream=True)
    with open(NAMES_ZIP_FILE, "wb") as f:
        shutil.copyfileobj(response.raw, f)

    with gzip.open("name.basics.tsv.gz", "rb") as f_in:
        with open("name.basics.tsv", "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

df_actors = pd.read_csv("name.basics.tsv", sep="\t", low_memory=False)


selected_columns = [
    "nconst",
    "primaryName",
    "birthYear",
    "primaryProfession"
]
clean_df = df_actors[selected_columns].head(10)
engine = create_engine(os.getenv("DATABASE_URL"))
try:
    table_name = "actors"
    clean_df.to_sql(table_name,engine, if_exists="replace", index=False)
    table = pd.read_sql("SELECT * FROM actors LIMIT 10", engine)
    logging.info(f"\n{table}") 
except Exception as e:
    logging.error("Exception:", e)
finally:
    engine.dispose()

2025-12-06 15:09:52,909 - INFO - 
      nconst      primaryName birthYear                  primaryProfession
0  nm0000001     Fred Astaire      1899       actor,miscellaneous,producer
1  nm0000002    Lauren Bacall      1924   actress,miscellaneous,soundtrack
2  nm0000003  Brigitte Bardot      1934  actress,music_department,producer
3  nm0000004     John Belushi      1949      actor,writer,music_department
4  nm0000005   Ingmar Bergman      1918              writer,director,actor
5  nm0000006   Ingrid Bergman      1915        actress,producer,soundtrack
6  nm0000007  Humphrey Bogart      1899       actor,producer,miscellaneous
7  nm0000008    Marlon Brando      1924              actor,director,writer
8  nm0000009   Richard Burton      1925            actor,producer,director
9  nm0000010     James Cagney      1899            actor,director,producer


In [ ]:
!docker-compose down -v